In [1]:
import requests, json
from urllib.parse import quote
import folium
import pandas as pd
import numpy as np

In [2]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [3]:
area = pd.read_csv('data/경기도청사및출장소현황.csv',encoding='euc-kr')
area.head()

,집계일자,시군명,구분명,전화번호안내,소재지우편번호,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도
0,2023-03-08,안성시,안성시청,031-678-2114,17586,경기도 안성시 시청길 25,경기도 안성시 봉산동 31-3번지,37.007357,127.280313
1,2023-03-08,안성시,고삼면,031-678-3980,17505,경기도 안성시 고삼면 고삼호수로 21,경기도 안성시 고삼면 가유리 287-11번지,37.082693,127.263456
2,2023-03-08,안성시,공도읍,031-678-3610,17560,경기도 안성시 공도4로 8,경기도 안성시 공도읍 만정리 793-1번지,37.001014,127.172515
3,2023-03-08,안성시,보개면,031-678-3680,17509,경기도 안성시 보개면 보개원삼로 220,경기도 안성시 보개면 불현리 42-1번지,37.023016,127.290858
4,2023-03-08,안성시,금광면,031-678-3710,17536,경기도 안성시 금광면 금광오산로 228,경기도 안성시 금광면 내우리 20-9번지,36.997233,127.317531


In [4]:
area = area[area.구분명.str.contains('도청') | area.구분명.str.contains('시청') | area.구분명.str.contains('구청')]
area.head()

,집계일자,시군명,구분명,전화번호안내,소재지우편번호,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도
0,2023-03-08,안성시,안성시청,031-678-2114,17586,경기도 안성시 시청길 25,경기도 안성시 봉산동 31-3번지,37.007357,127.280313
17,2023-02-15,안산시,단원구청,031-481-6000,15396,경기도 안산시 단원구 중앙대로 685 (초지동),경기도 안산시 단원구 초지동 666-2번지,37.320905,126.815245
22,2023-02-15,안산시,안산시청,031-481-2000,15335,경기도 안산시 단원구 화랑로 387 (고잔동),경기도 안산시 단원구 고잔동 515번지,37.321956,126.830878
42,2023-02-15,안산시,상록구청,031-481-5000,15585,경기도 안산시 상록구 석호로 110 (사동),경기도 안산시 상록구 사동 1585번지,37.300692,126.846368
52,2023-02-14,군포시,군포시청,031-392-3000,15829,경기도 군포시 청백리길 6,경기도 군포시 금정동 844번지,37.361254,126.936150


In [5]:
with open('data/kakaoapikey.txt') as f:
    kakao_key = f.read()

In [6]:
base_url = "https://dapi.kakao.com/v2/local/search/address.json"
addr = '경기도 수원시 영통구 도청로 30 (이의동)'
url = f'{base_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EA%B2%BD%EA%B8%B0%EB%8F%84%20%EC%88%98%EC%9B%90%EC%8B%9C%20%EC%98%81%ED%86%B5%EA%B5%AC%20%EB%8F%84%EC%B2%AD%EB%A1%9C%2030%20%28%EC%9D%B4%EC%9D%98%EB%8F%99%29'

In [7]:
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [8]:
lat_list1, lng_list1 = [],[]
for i in area.index:
    url = f'{base_url}?query={quote(area.소재지도로명주소[i])}'
    result = requests.get(url, headers=header).json()
    lat_list1.append(float(result['documents'][0]['y']))
    lng_list1.append(float(result['documents'][0]['x']))

In [10]:
area['위도'] = lat_list1
area['경도'] = lng_list1
area.head(3)

,집계일자,시군명,구분명,전화번호안내,소재지우편번호,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도,위도,경도
0,2023-03-08,안성시,안성시청,031-678-2114,17586,경기도 안성시 시청길 25,경기도 안성시 봉산동 31-3번지,37.007357,127.280313,37.008308,127.279210
17,2023-02-15,안산시,단원구청,031-481-6000,15396,경기도 안산시 단원구 중앙대로 685 (초지동),경기도 안산시 단원구 초지동 666-2번지,37.320905,126.815245,37.319777,126.811755
22,2023-02-15,안산시,안산시청,031-481-2000,15335,경기도 안산시 단원구 화랑로 387 (고잔동),경기도 안산시 단원구 고잔동 515번지,37.321956,126.830878,37.322557,126.832714


In [19]:
area['시군구'] = area['구분명'].str.contains('시청')


0     안성시청
17    안성시청
22    안성시청
42    안성시청
52    안성시청
Name: 시군구, dtype: object

In [20]:
import folium
map = folium.Map(location=[area.위도.mean(), area.경도.mean()],zoom_start=12)
for i in area.index:
    folium.Marker(
        location=[area.위도[i],area.경도[i]],
        popup=folium.Popup(area.전화번호안내[i], max_width=200),
        tooltip=area.구분명[i]
    ).add_to(map)

map